In [1]:
import os
import re
import nltk
from pdfminer.high_level import extract_pages
from pdfminer.high_level import LTPage
from nltk.corpus import wordnet as wn
from pdfminer.layout import LTTextContainer, LTTextLineVertical, LTTextBoxHorizontal, LTTextBox, Rect
import wordninja
from pdf2image import convert_from_path
import numpy as np
import cv2 

In [2]:
class SpanishLanguageModel(wordninja.LanguageModel):
    def __init__(self, word_file):
        super().__init__(word_file)
        self.SPLIT_RE = re.compile("[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑ']+")
    
    def split(self, s):
        l = [self._split(x) for x in self.SPLIT_RE.split(s)]
        return [item for sublist in l for item in sublist]
wordninja.DEFAULT_LANGUAGE_MODEL = SpanishLanguageModel("words.txt.gz")

def separate_words(text):
    sep = wordninja.split(text)
    return " ".join(sep)

In [3]:
os.makedirs("data", exist_ok=True)
articles = [os.path.join("data", art) for art in os.listdir("data")]
articles

['data\\000.pdf',
 'data\\001.pdf',
 'data\\002.pdf',
 'data\\003.pdf',
 'data\\004.pdf',
 'data\\101404-Texto del artículo-379745-1-10-20230814.pdf',
 'data\\103822-Texto del artículo-398336-1-10-20240109.pdf',
 'data\\103827-Texto del artículo-398338-1-10-20240109.pdf',
 'data\\108896-Texto del artículo-430136-1-10-20240723.pdf',
 'data\\110685-Texto del artículo-444650-1-10-20241017.pdf',
 'data\\113135-Texto del artículo-461992-1-10-20250121.pdf',
 'data\\113136-Texto del artículo-461995-1-10-20250121.pdf',
 'data\\130-Texto del artículo en fichero de Microsoft Word o LibreOffice (necesario)-134-1-10-20090331.pdf',
 'data\\32502-Texto del artículo-103690-1-10-20150102 (1).pdf',
 'data\\32502-Texto del artículo-103690-1-10-20150102.pdf',
 'data\\32511-Texto del artículo-103699-1-10-20150102.pdf',
 'data\\32519-Texto del artículo-103707-1-10-20150102.pdf',
 'data\\32576-Texto del artículo-103765-1-10-20150102.pdf',
 'data\\32585-Texto del artículo-103775-1-10-20150102.pdf',
 'data\\3

In [38]:
def extract_possible_footnote_line(article_path: str, page: LTPage, page_number: int, DPI: int = 100) -> list[dict[str, float]]:
    page_w, page_h = page.bbox[2], page.bbox[3]
    raw_image = convert_from_path(article_path, dpi=DPI, first_page=page_number+1, last_page=page_number+1)[0]
    img = np.array(raw_image)
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_height, img_width = gray.shape
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
    morph = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    edges = cv2.Canny(morph, 50, 200, apertureSize=3)

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=20)
    if lines is None:
        return None

    pdfminer_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # Escalar a coordenadas PDFMiner
        y1_pdf = (y1 / img_height) * page_h
        y2_pdf = (y2 / img_height) * page_h

        pdfminer_lines.append({
            "x1": x1,
            "y1": page_h - y1_pdf,
            "x2": x2,
            "y2": page_h - y2_pdf
        })

    threshold_y = page_h * 0.35
    filtered_lines = [line for line in pdfminer_lines if line["y1"] <= threshold_y]
    filtered_lines.sort(key=lambda line: line["y1"], reverse=True)
    return filtered_lines[0] if filtered_lines else None

**arreglar**:

etno-\ngráfico -> etnográfico -> .replace("-\n", "")

**tomar en cuenta toda la pagina hasta donde aparece el primer patron de pie de pagina**

**quitar**:
- citas como (Banks, 2001)

**arreglar**:
- hay conjunto de palabras que aparecen juntos, por ejemplo, lacercania en lugar de 
la cercania (art4, pag3)

In [39]:
def extract_page(pdf_path, page):
    pages = extract_pages(pdf_path)
    for i, page_layout in enumerate(pages):
        if i != page - 1: continue
        return page_layout
    return None

In [75]:
# regex stop conditions
references_re = r"(?i)^\s*(referencias|bibliografía citada|bibliografia citada|bibliografía|bibliografia|citas|fuentes|referencias bibliograficas|referencias bibliográficas)\s*$"

# regex delete
version_logs = r"(?i)(recibido|aceptado|publicado|(segunda|tercera|cuarta|quinta|sexta) versión)[,:]?\s*(el\s*)?(\d{1,2}(\s*de)?\s*[a-záéíóú]+(\s*de\s*\d{4})?|\d{1,2}/\d{1,2}/\d{4})(\s*\.\s*)?"
footnote_patterns = r"(?m)^(?:\d+|[*†‡¹²³⁴⁵⁶⁷⁸⁹]+|\[\d+\])\s+.*$"

footnote_references = r"([.,;:!?])([0-9]+)"
duplicated_spaces = r"\s+"

# recognizing regex
abstract = r"Palabras clave|Keywords|Resumen|Abstract|Key words|Palabrasclave|Keyword|Palabra clave|Palabras claves|Keywords|Sumario|Síntesis|Sintesis|Sinopsis"

# parentesis y llaves
parentheses_pattern = r'\(' + \
                      r'(?:' + \
                      r'(?:[^;)]+)' + \
                      r'(?:; [^;)]+)*' + \
                      r')\)'
brackets_pattern = r'\[' + \
                      r'(?:' + \
                      r'(?:[^;\]]+)' + \
                      r'(?:; [^;\]]+)*' + \
                      r')\]'

In [76]:
class ArticleExtractedData:
    def __init__(self, pdf_path: str):
        self.pdf_path: str = pdf_path
        self.headers: set[str] = set()
        self.footers: set[str] = set()
        self.footnotes_lines: list[dict[str, float]] = []
        self.start_page: int = 0

def extract_data_article(pdf_path) -> ArticleExtractedData:
    data = ArticleExtractedData(pdf_path)
    headers_counter: map[str, int] = {}
    footers_counter: map[str, int] = {}
    found_page = None

    for i, page in enumerate(extract_pages(pdf_path)):
        height = page.bbox[3]
        cutoff = height * 0.15

        footnote_line = extract_possible_footnote_line(pdf_path, page, i)
        data.footnotes_lines.append(footnote_line)

        for element in page:
            if not isinstance(element, (LTTextBox, LTTextBox, LTTextContainer)):
                continue
            txt = element.get_text().strip()
            if element.bbox[3] <= cutoff:
                footers_counter[txt] = footers_counter.get(txt, 0) + 1
            if element.bbox[1] >= height - cutoff:
                headers_counter[txt] = headers_counter.get(txt, 0) + 1
            
            # Stop searching for abstract after 3 pages
            if found_page is not None or i > 2:
                continue

            text_semiclean = re.sub(duplicated_spaces, " ", txt).strip()
            if re.search(abstract, text_semiclean, re.IGNORECASE):
                found_page = i


    data.headers = {k for k, v in headers_counter.items() if v > 1 and len(k) > 0}
    data.footers = {k for k, v in footers_counter.items() if v > 1 and len(k) > 0}
    if found_page is not None:
        data.start_page = found_page
    return data

In [77]:
def remove_accents(text: str) -> str:
    return text.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")

In [78]:
separate_words("espaciospúblicosylasautónomas")

'espacios público s y lasa u t ó n o m a s'

**los footnote references se llevan los años, en textos como: en 1592. Aquello [...]**

In [112]:
def check_stop_conditions(text) -> bool:
	return any([
		re.search(references_re, text, flags=re.MULTILINE),
	])

def clean_unwanted_patterns(text) -> str:
	# Eliminar patrones como "Recibido el 12 de marzo de 2021."
    text = re.sub(version_logs, "", text).strip()
    text = re.sub(parentheses_pattern, "", text).strip()
    text = re.sub(brackets_pattern, "", text).strip()
    text = re.sub(footnote_references, r"\1", text).strip()

	# text = re.sub(cites_pattern, "", text).strip()
	# text = re.sub(footnote_patterns, "", text, flags=re.MULTILINE).strip()

	# Eliminar las referencias de las notas al pie y reemplazarlas por un punto
	# text = re.sub(footnote_references, ".", text).strip()

    return text

def avoid_unwanted_texts(text) -> str:
	return text

def clean_page(text) -> str:
    text = text.replace("-\n", "")
    text = text.replace("\n", " ")
    text = text.replace("«", "").replace("»", "").replace("“", "").replace("”", "").replace("‘", "").replace("’", "")
    text = text.replace('"', "").replace("'", "")
    text = re.sub(duplicated_spaces, " ", text).strip()
    return text.lower()

# def corrections(text: str) -> str:
#     words = text.split()
#     corrected = []
#     for word in words:
#         if len(word) < 3:
#             continue
#         corrected.append(separate_words(word))
#     return " ".join(corrected)

# (str, bool) -> (text, continue?)
def extract_page_text(page: LTPage, page_number: int, data: ArticleExtractedData) -> tuple[str, bool]:
    text = ""
    status = True
    if page_number <= data.start_page:
        return text, status

    for element in page:
        if not isinstance(element, (LTTextBoxHorizontal, LTTextBox, LTTextContainer)):
            continue

        _, __, ___, y2 = element.bbox
        footnote_line = data.footnotes_lines[page_number]
        if y2 <= (footnote_line["y1"] if footnote_line is not None else -1):
            break

        txt = element.get_text().strip()
        if txt.isdigit() or txt in data.headers or txt in data.footers:
            continue

        txt = avoid_unwanted_texts(txt)
        if check_stop_conditions(txt):
            status = False
            break

        text += txt + "\n"

    text = clean_unwanted_patterns(text)
    text = clean_page(text)
    # text = corrections(text)
    return text, status

In [105]:
art = articles[0]
data = extract_data_article(art)
page = extract_page(art, 28)
extract_page_text(page, 27, data)

'inmigrante se hereda y, al mismo tiempo, se contribuye a hacer patente que la supuesta pureza cultural y la idea de una esencia primigenia no son las únicas vías de construcción de identidades étnicas. son cinco ideas desde la reflexión, para afrontar las consecuencias perniciosas del corsé estructural con el que deben bregar quienes se han desplazado, quienes se supone que nunca lo han hecho y quienes sin haberse desplazado son vistos como si lo hubieran hecho, en el esfuerzo por construir un futuro que invite algo al optimismo.'

In [106]:
page = extract_page(articles[1], 3)
for element in page:
    print(element)

<LTTextBoxHorizontal(0) 131.553,617.258,367.336,625.258 'REFORMA, ORDEN Y CONCIERTO EN EL PERÚ DEL SIGLO XVII\n'>
<LTTextBoxHorizontal(1) 85.038,387.647,413.873,601.046 'constantemente  con  los  otros  miembros  de  la  élite  de  poder  en  materias\nreferentes  a  aumento  de  impuestos,  envío  de  remesas  y  colaboración\ncomercial.  Mantener  el  equilibrio  no  fue  tarea  fácil  y,  de  hecho,  hubo\ncoyunturas muy tensas entre los diversos actores sociales, sobre todo cuan-\ndo  se  intentó  forzar  a  los  reinos  americanos  a  realizar  mayores  aportes\npecuniarios a la península. Los «beneméritos» nunca cejaron en sus inten-\ntos de obtener los puestos que los propios virreyes repartían entre sus cria-\ndos. La Iglesia, por su parte, se mantuvo firme en defender los privilegios\nconcedidos desde el siglo XVI, que la favorecían con la percepción de diez-\nmos, inmunidad, cesión de tierras y beneficios fiscales. La administración\nfue ineficiente y progresivamente ciertas 

In [107]:
import pdfplumber

def detect_lines_in_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            lines = page.lines  # Extrae todas las líneas de la página

            # Filtrar líneas horizontales (y0 ≈ y1 indica una línea horizontal)
            horizontal_lines = [line for line in lines if abs(line["y0"] - line["y1"]) < 1]

            if horizontal_lines:
                print(f"Página {i+1}: {len(horizontal_lines)} ({(page.width, page.height)}) línea(s) horizontal(es) detectada(s).")
                for line in horizontal_lines:
                    print(f"  - Coordenadas: {line}")

detect_lines_in_pdf(articles[2])


In [108]:
import pdfquery
pdf = pdfquery.PDFQuery(articles[0])
pdf.load()
lines = pdf.pq('LTLine')
print(len(lines))
for line in lines:
    print(line.get('bbox'))

31
[92.7, 744.18, 519.36, 744.18]
[191.7, 317.16, 420.3, 317.16]
[191.7, 91.08, 420.3, 91.08]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]


# Preprocesado

In [109]:
articles = [os.path.join("data", art) for art in os.listdir("data")]
len(articles)

36

In [117]:
def extract_pdf_text(pdf_path: str) -> str:
    text = ""
    data = extract_data_article(pdf_path)
    for i, page in enumerate(extract_pages(pdf_path)):
        page_text, can_continue = extract_page_text(page, i, data)
        text = f"{text} {page_text}"
        if not can_continue:
            break
    return text.strip()

In [ ]:
for article in articles:
    print(extract_pdf_text(article))

1. introducción en el reflujo de la esperanza y la desesperación, la respuesta defensiva y el conformismo, la resistencia y la acomodación, las auto-identificaciones colectivas emergen al mismo tiempo como huella de los factores estructurales y como una manifestación de la agencia humana que se obstina en lidiar contra aquellas estructuras . ste artículo es una propuesta de revisión de trabajos que se han desarrollado en estados unidos y la unión europea sobre las llamadas segundas generaciones, que es de prever que en los e próximos años constituya un lugar común en los debates sociales y académicos alrededor de los procesos migratorios. dicho esfuerzo de revisión se propone persiguiendo dos grandes objetivos. en primer lugar, ver cuáles han sido los planteamientos teóricos, los sujetos de estudio y las conclusiones de algunos de los principales estudios realizados en la materia de los casos a tener en cuenta. en especial se enfocará qué es lo que los distintos autores han entendido c